In [2]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

We shall use the ImageDataGenerator class to feed in the training and validation data to the model. This class can also be used to generate augmented data.

To know more about ImageDataGenerator class, visit https://keras.io/preprocessing/image/#imagedatagenerator-class

In [3]:
train_datagenerator = ImageDataGenerator(rescale=1./255)
test_datagenerator = ImageDataGenerator(rescale=1./255)

train_datagenerator = train_datagenerator.flow_from_directory(
    'train',
    target_size=(128,128),
    batch_size=40,
    class_mode='binary')

test_datagenerator = test_datagenerator.flow_from_directory(
    'test',
    target_size=(128,128),
    batch_size=10,
    class_mode='binary')

Found 40 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


Our model will have 3 Convolution2D layers. You can increse or decrease as per your needs.

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3),padding='same', activation='relu', input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D((2,2),2),
    
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),     
     
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),   
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [5]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0

We shall use Adam optimizer with a learning rate of 0.001 (hyperparameter). We choose 'binary_crossentropy' loss as our model is a binary calssifier (i.e, we have only 2 classes)

In [6]:
model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(0.001),
             metrics=['accuracy'])

At the end of each epoch we can check if the model has reached the required accuracy and terminate the training.

In [7]:
DESIRED_ACCURACY = 0.85

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if((logs.get('acc')>DESIRED_ACCURACY) and (logs.get('val_acc')>DESIRED_ACCURACY )):
      print("\nReached 85% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

To know more about fit_generator visit https://keras.io/models/model/#fit_generator

In [8]:
model.fit_generator(
    train_datagenerator,
    epochs=100,
    validation_data = test_datagenerator,
    callbacks = [callbacks]
    )

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 0.6918 - acc: 0.6000 - val_loss: 1.4510 - val_acc: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 164ms/step - loss: 1.0046 - acc: 0.5000 - val_loss: 1.3116 - val_acc: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 169ms/step - loss: 1.3999 - acc: 0.5000 - val_loss: 0.7607 - val_acc: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 164ms/step - loss: 0.7762 - acc: 0.5000 - val_loss: 0.6825 - val_acc: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 164ms/step - loss: 0.6200 - acc: 0.5000 - val_loss: 0.6941 - val_acc: 0.8000
Epoch 6/100
1/1 [==============================] - 0s 170ms/step - loss: 0.5829 - acc: 0.9500 - val_loss: 0.6923 - val_acc: 0.8000
Epoch 7/100
1/1 [==============================] - 0s 165ms/step - loss: 0.5522 - acc: 0.8250 - val_loss: 0.6907 - val_acc: 0.8000
Epoch 8/100
1/1 [==============================] - 0s 164ms/step - loss: 0.5020 - acc:

1/1 [==============================] - 0s 165ms/step - loss: 3.9736e-07 - acc: 1.0000 - val_loss: 3.9085 - val_acc: 0.8000
Epoch 63/100
1/1 [==============================] - 0s 164ms/step - loss: 3.8953e-07 - acc: 1.0000 - val_loss: 3.9172 - val_acc: 0.8000
Epoch 64/100
1/1 [==============================] - 0s 185ms/step - loss: 3.8259e-07 - acc: 1.0000 - val_loss: 3.9251 - val_acc: 0.8000
Epoch 65/100
1/1 [==============================] - 0s 196ms/step - loss: 3.7635e-07 - acc: 1.0000 - val_loss: 3.9322 - val_acc: 0.8000
Epoch 66/100
1/1 [==============================] - 0s 165ms/step - loss: 3.7066e-07 - acc: 1.0000 - val_loss: 3.9387 - val_acc: 0.8000
Epoch 67/100
1/1 [==============================] - 0s 165ms/step - loss: 3.6540e-07 - acc: 1.0000 - val_loss: 3.9446 - val_acc: 0.8000
Epoch 68/100
1/1 [==============================] - 0s 182ms/step - loss: 3.6049e-07 - acc: 1.0000 - val_loss: 3.9499 - val_acc: 0.8000
Epoch 69/100
1/1 [==============================] - 0s 187ms/

In [10]:
model.save('mymodel.h5')